# Optimal Generator Dispatch

## 1 - Problem Description

Transmission system operators around the world typically employ some form of mathematical programming to plan and operate power grids. In the United Kingdom, each day is split into six discrete 4-hour blocks, called *Electricity Forward Agreement* (EFA) blocks, each with an anticipated total demand. In this problem, you will determine which of the available generator units should be dispatched to meet said demand in each of the EFA blocks so as to minimize total costs. In addition, you must consider the necessity for a reserve capacity, that is, the chosen units need the capability to increase their output while still adhering to their specified operating limits. This is essential to handle a situation where real demand surpasses anticipated demand.

test

## 2 - Available Data
Several files are available that contain the required the data to formulate the problem:
- Anticpated demand in each EFA block: `./demand.csv`
- Generator unit types, number of each type available and the operating limits of each: `./generators.csv`
- The costs associated with each generator unit type, including a fixed base cost for running the unit (GBP), a marginal cost for generation above its minimum output (GBP/MW), and a fixed start up cost for switching a generator unit on (GBP): `./costs.csv`

To consdider reserve, assume the set of dispatched generator units should be able to produce 115% of predicted demand. Assume that there are already 5 generator units of each time running before the start of the first EFA block.

## 3 - Problem Formulation

### 3.1 - Decision Variables
- Number of generator units of type $t$ that are on during EFA block $b$: $N^{\text{gen}}_{t,b}$
- Power output from all generators of type $t$ during time period $p$: $P_{t,b}$
- Number of generators of type $t$ that startup in EFA block $b$: $N^{\text{startups}}_{t,b}$

### 3.2 - Objective

\begin{equation}
\textbf{minimize} 
\sum_{t, b}{C^{\textrm{base}}_t* N^{\text{gen}}_{t,b}} + \sum_{t, b}{C^{\text{running}}_t*(P_{t,b} - P^{\text{min}}_t}) + \sum_{t, b}{C^{\text{startup}}_t* N^{\text{startups}}_{t,b}}
\end{equation}

### 3.3 - Constraints
- The power output of each generator unit must satisfy the specified operating limits.
- In each EFA block, the dispatched generator units must not violate reserve requirements.
- The dispatched generator units must also provide enough power to satisfy the anticipated demand.
- The total number of generator units on during each EFA block cannot exceed the number available.

## 4 - Tasks
1. Import the required data.
2. Complete the mathematical formulation and implement the corresponding program in Python with Gurobi.
3. Evaluate the following:
    - The total cost of satisyfing the anticipated demand.
    - In each EFA block, the number of generator units that are active, as well as the number that must be switched on.

## 5 - Solution

### Imports

In [5]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import pandas as pd

### 1. Import Data

In [10]:
costs = pd.read_csv('costs.csv')
demand = pd.read_csv('demand.csv')
generators = pd.read_csv('generators.csv')

In [7]:
costs
# Base: fixed base cost of running the unit GBP
# Running: marginal cost for generating above its minimum output GBP/MW
# Startup: fixed switching a generator unit on GBP

,base,running,startup
0,1000,2.0,2000
1,2600,1.3,1000
2,3000,3.0,500


- Anticpated demand in each EFA block: `./demand.csv`
- Generator unit types, number of each type available and the operating limits of each: `./generators.csv`

In [12]:
demand
# Power: the demanding power for each block
# efa_block: the block we are

,power,efa_block
0,15000,1
1,30000,2
2,25000,3
3,25000,4
4,40000,5
5,27000,6


In [11]:
generators
# num_of generator: the number of generator we have for each type
# min_power: minimum power to be used
# max_power: maximum power can be used

,type,num_generators,min_power,max_power
0,1,12,850,2000
1,2,10,1250,1750
2,3,5,1500,4000


### 2. Mathematical Formulation

Dispatched generators must produce 115% of the predicted demand:

In [13]:
reserve = 1.15

In [ ]:
# Building model

model = gp.Model("Gurobi_exercise") 

# Variables

nG = model.addVar(vtype=GRB.INTEGER, name="numGenerators")
PO = model.addVar(vtype=GRB.INTEGER, name="PowerOutput")
nGs = model.addVar(vtype=GRB.INTEGER, name="numGeneratorsStarters")

# Constrains

